In [38]:
# Install MySQL connector
!pip install mysql-connector-python

In [39]:
import numpy as np
import mysql
import mysql.connector as msql
from mysql.connector import Error
from datetime import datetime
import pandas as pd

In [40]:
def get_db_connection(userName, Password):
  # establish database connection
  try:
    conn = msql.connect(host='db4free.net',database='education_nz' ,user=userName,
                          password=Password)
  except Error as e:
      print("Error while connecting to MySQL now ", e)

  return conn

In [41]:
def set_connection_close(conn):
  # Close the cursor and connection
  if conn.is_connected():
    conn.close()

In [42]:
def log_error(conn, errorTitle, error_message):

  # Get the current date and time
  now = datetime.now()

  # Format the current date and time as a string in MySQL datetime format
  dt_string = now.strftime('%Y-%m-%d %H:%M:%S')

  try :
    if conn.is_connected():
      cursor = conn.cursor()

      # SQL query to insert data into the 'error' table
      insert_query = 'INSERT INTO education_nz.error (title, description, creation_datetime) VALUES (%s, %s, %s)'
      values =(errorTitle, error_message, dt_string )

      cursor.execute(insert_query, values)
      conn.commit()

  except Error as e:
      print("Error while inserting to error table", e)
  finally:
    # Close the cursor and connection
    if conn.is_connected():
      cursor.close()

In [43]:
def get_courseDescriptor_Details(conn, version_id, page_start, page_end):

  # Initialize an empty DataFrame
  df = pd.DataFrame()

  try:
      # Create a cursor to execute queries
      if conn.is_connected():
        cursor = conn.cursor(dictionary=True, buffered=True)

        # Define the MySQL query
        query = """
              SELECT
                  cor.versionID,
                  cor.pageNo,
                  cor.courseID,
                  cor.code,
                  cor.title,
                  cor.credit,
                  cor.level,
                  cor.self_directed_learning_hrs,
                  cor.tutor_directed_learning_hrs,
                  cor.aim,
                  comp.completion,
                  ou.outcome,
                  pre.prerequisite_code,
                  pre.prerequisite_title,
                  ass.assessment_method,
                  ass.assessment_weight,
                  ass.assessment_outcome
              FROM
                  course cor
              LEFT JOIN
                  (SELECT comp.courseID, GROUP_CONCAT(comp.criteria ORDER BY comp.completionID, ' ') AS completion
                  FROM completion comp GROUP BY comp.courseID) comp
              ON
                  cor.courseID = comp.courseID
              LEFT JOIN
                  (SELECT ou.courseID, GROUP_CONCAT(ou.outcome ORDER BY ou.outcomeID, ' ') AS outcome
                  FROM outcome ou GROUP BY ou.courseID) ou
              ON
                  cor.courseID = ou.courseID
              LEFT JOIN
                  (SELECT pre.courseID, GROUP_CONCAT(pre.code ORDER BY pre.prerequisiteID, ' ') AS prerequisite_code,
                                      GROUP_CONCAT(pre.title ORDER BY pre.prerequisiteID, ' ') AS prerequisite_title
                  FROM prerequisite pre GROUP BY pre.courseID) pre
              ON
                  cor.courseID = pre.courseID
              LEFT JOIN
                  (SELECT ass.courseID, GROUP_CONCAT(ass.method ORDER BY ass.assessmentID, ' ') AS assessment_method,
                                        GROUP_CONCAT(ass.weight_precentage ORDER BY ass.assessmentID, ' ') AS assessment_weight,
                                        GROUP_CONCAT(ass_out.outcomeID_value ORDER BY ass_out.assessmentID, ' ') AS assessment_outcome
                  FROM assessment ass
                  LEFT JOIN
                      (SELECT ass_out.assessmentID, GROUP_CONCAT(ass_out.outcomeID_value, ' ') AS outcomeID_value
                        FROM assessment_outcome ass_out GROUP BY ass_out.assessmentID) ass_out
                  ON
                      ass.assessmentID = ass_out.assessmentID
                  GROUP BY ass.courseID) ass
              ON
                  cor.courseID = ass.courseID
              WHERE
                  cor.versionID = %s AND cor.pageNo BETWEEN %s AND %s
              ORDER BY
                  cor.pageNo;
        """

        # Execute the query with parameters
        cursor.execute(query, (version_id, page_start, page_end))

        # Fetch all results
        results = cursor.fetchall()

        # Get column names
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame from the results
        df = pd.DataFrame(results, columns=column_names)



  except Error as e:
      title = "Error while fetching course descriptor details"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()


  return df


In [44]:
if __name__ == '__main__':
  # Example usage
  version_id = 5
  page_start = 41
  page_end = 50

  dbConnection = get_db_connection('whitireia_admin', 'weltec#2023')

  query_results = get_courseDescriptor_Details(dbConnection, version_id, page_start, page_end)

  # Print or process the results as needed
  #for result in query_results:
      #print(result)

  print(query_results.head())

  set_connection_close(dbConnection)

   versionID  pageNo  courseID    code  \
0          5      41         6  IT5117   
1          5      42         7  IT5118   
2          5      43         8  IT5119   
3          5      44         9  IT5120   
4          5      45        10  IT5121   

                                              title  credit  level  \
0                                HARDWARE SERVICING      15      5   
1                             PROFESSIONAL PRACTICE      15      5   
2                              IT TECHNICAL SUPPORT      15      5   
3                                 SERVER MANAGEMENT      15      5   
4  FUNDAMENTALS OF PROGRAMMING AND PROBLEM -SOLVING      15      5   

   self_directed_learning_hrs  tutor_directed_learning_hrs  \
0                          65                           85   
1                          65                           85   
2                          65                           85   
3                          65                           85   
4               

In [45]:
def insert_comparison_version(conn, name, description, version_1, version_2):

  try:
      if conn.is_connected():
        cursor = conn.cursor()

        # Define the MySQL query for insertion
        insert_query = """
            INSERT INTO comparison_version (comparison_name, comparison_description, comparison_version_1, comparison_version_2, created_datetime)
            VALUES (%s, %s, %s, %s, %s)
        """

        # Get the current datetime
        current_datetime = datetime.now()

        # Execute the insertion query
        cursor.execute(insert_query, (name, description, version_1, version_2, current_datetime))

        # Commit the changes
        conn.commit()

        # Get the last inserted ID (comparison_versionID)
        comparison_version_id = cursor.lastrowid

        return comparison_version_id

  except Error as e:
      title = "Error while inserting comparison version"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()


In [46]:
# if __name__ == '__main__':
#   # Example usage
#   comparison_name = 'YourComparisonName'
#   comparison_description = 'YourComparisonDescription'
#   version_1 = 1
#   version_2 = 2

#   inserted_id = insert_comparison_version(comparison_name, comparison_description, version_1, version_2)

#   if inserted_id is not None:
#       print(f"Comparison Version ID {inserted_id} inserted successfully.")
#   else:
#       print("Failed to insert comparison version.")

In [47]:
def insert_course_comparison(conn,
    comparison_versionID, pageNo,
    code_ratio, code_version1, code_version2,
    title_ratio, title_version1, title_version2,
    credit_ratio, credit_version1, credit_version2,
    level_ratio, level_version1, level_version2,
    self_directed_learning_ratio, self_directed_learning_version1, self_directed_learning_version2,
    tutor_directed_learning_ratio, tutor_directed_learning_version1, tutor_directed_learning_version2,
    aim_ratio, aim_version1, aim_version2,
    completion_ratio, completion_version1, completion_version2,
    outcome_ratio, outcome_version1, outcome_version2,
    prerequisite_code_ratio, prerequisite_code_version1, prerequisite_code_version2,
    prerequisite_title_ratio, prerequisite_title_version1, prerequisite_title_version2,
    assessment_method_ratio, assessment_method_version1, assessment_method_version2,
    assessment_weight_ratio, assessment_weight_version1, assessment_weight_version2,
    assessment_outcome_ratio, assessment_outcome_version1, assessment_outcome_version2
):


  try:
      if conn.is_connected():
        cursor = conn.cursor()

        # Define the MySQL query for insertion
        insert_query = """
            INSERT INTO course_comparison (
                comparison_versionID, pageNo,
                code_ratio, code_version1, code_version2,
                title_ratio, title_version1, title_version2,
                credit_ratio, credit_version1, credit_version2,
                level_ratio, level_version1, level_version2,
                self_directed_learning_ratio, self_directed_learning_version1, self_directed_learning_version2,
                tutor_directed_learning_ratio, tutor_directed_learning_version1, tutor_directed_learning_version2,
                aim_ratio, aim_version1, aim_version2,
                completion_ratio, completion_version1, completion_version2,
                outcome_ratio, outcome_version1, outcome_version2,
                prerequisite_code_ratio, prerequisite_code_version1, prerequisite_code_version2,
                prerequisite_title_ratio, prerequisite_title_version1, prerequisite_title_version2,
                assessment_method_ratio, assessment_method_version1, assessment_method_version2,
                assessment_weight_ratio, assessment_weight_version1, assessment_weight_version2,
                assessment_outcome_ratio, assessment_outcome_version1, assessment_outcome_version2
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Execute the insertion query
        cursor.execute(insert_query, (
            comparison_versionID, pageNo,
            code_ratio, code_version1, code_version2,
            title_ratio, title_version1, title_version2,
            credit_ratio, credit_version1, credit_version2,
            level_ratio, level_version1, level_version2,
            self_directed_learning_ratio, self_directed_learning_version1, self_directed_learning_version2,
            tutor_directed_learning_ratio, tutor_directed_learning_version1, tutor_directed_learning_version2,
            aim_ratio, aim_version1, aim_version2,
            completion_ratio, completion_version1, completion_version2,
            outcome_ratio, outcome_version1, outcome_version2,
            prerequisite_code_ratio, prerequisite_code_version1, prerequisite_code_version2,
            prerequisite_title_ratio, prerequisite_title_version1, prerequisite_title_version2,
            assessment_method_ratio, assessment_method_version1, assessment_method_version2,
            assessment_weight_ratio, assessment_weight_version1, assessment_weight_version2,
            assessment_outcome_ratio, assessment_outcome_version1, assessment_outcome_version2
        ))

        # Commit the changes
        conn.commit()

  except Error as e:
      title = "Error while inserting course comparison"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()



In [48]:
# if __name__ == '__main__':
#   # Example usage
#   comparison_versionID = 1
#   pageNo = 42
#   # ... (add other values for each column)
#   # For simplicity, you can replace the ellipsis (...) with actual values for each column.

#   inserted_id = insert_course_comparison(
#       comparison_versionID, pageNo,
#       0.75, 'version1_code', 'version2_code',
#       0.80, 'version1_title', 'version2_title',
#       # ... (add other values for each column)
#   )

#   if inserted_id is not None:
#       print(f"Course Comparison ID {inserted_id} inserted successfully.")
#   else:
#       print("Failed to insert course comparison.")
